In [7]:
# --- London Police UK cleaner (reads directly from the folder) ---
import os, glob, math
import pandas as pd
from pathlib import Path

        

In [13]:
# === 1) POINT THIS TO FOLDER ===
# Use a raw string (r"...") so backslashes are handled correctly on Windows
BASE_DIR = r"C:\Users\hksai\OneDrive\Documents\Group_Porject_Doc\Data_Police_UK_ALL(2025)\2025-08"

In [14]:
# === 2) OUTPUT FILES (will be written in the current working directory) ===
OUT_CLEAN = "london_crime_clean.csv"
OUT_BAD = "rows_excluded.csv"


In [15]:
# === 3) CLEANING CONFIG ===
FORCE_FILTERS = ["metropolitan", "city-of-london"]
BBOX = {"lat_min": 51.28, "lat_max": 51.70, "lon_min": -0.51, "lon_max": 0.33}
CHUNKSIZE = 200_000

In [16]:
CATEGORY_MAP = {
    "violence and sexual offences": "Violence_Sexual",
    "anti-social behaviour": "Antisocial_Behavior",
    "vehicle crime": "Vehicle_Crime",
    "bicycle theft": "Theft_Bicycle",
    "theft from the person": "Theft_Personal",
    "shoplifting": "Theft_Retail",
    "other theft": "Theft_Other",
    "criminal damage and arson": "Criminal_Damage",
    "public order": "Public_Order",
    "drugs": "Drug_Offenses",
    "burglary": "Burglary",
    "robbery": "Robbery",
    "possession of weapons": "Weapons",
    "other crime": "Other"
}

SEVERITY = {
    "Violence_Sexual": 3.0, "Robbery": 2.8, "Burglary": 2.5, "Vehicle_Crime": 2.0,
    "Theft_Bicycle": 1.8, "Criminal_Damage": 1.5, "Public_Order": 1.3,
    "Drug_Offenses": 1.2, "Antisocial_Behavior": 1.0, "Theft_Other": 1.0,
    "Theft_Personal": 1.5, "Theft_Retail": 0.8, "Weapons": 2.6, "Other": 1.0
}


In [23]:
# === 4) HELPERS ===
def standardize_columns(df):
    return df.rename(columns={c: c.strip().replace(" ", "_").lower() for c in df.columns})

def within_bbox(lat, lon):
    return ((lat >= BBOX["lat_min"]) & (lat <= BBOX["lat_max"]) &
            (lon >= BBOX["lon_min"]) & (lon <= BBOX["lon_max"]))

def filter_forces(df):
    # Keep rows where either 'reported_by' or 'falls_within' mention the target forces
    cols = [c for c in ("reported_by","falls_within") if c in df.columns]
    if not cols:
        return df, pd.DataFrame()
    mask = df[cols[0]].str.lower().str.contains("|".join(FORCE_FILTERS), na=False)
    if len(cols) > 1:
        mask |= df[cols[1]].str.lower().str.contains("|".join(FORCE_FILTERS), na=False)
    return df[mask], df[~mask]
    
    # 5) FIND CSVs UNDER YOUR FOLDER
csv_files = sorted(glob.glob(os.path.join(BASE_DIR, "**", "*.csv"), recursive=True))
if not csv_files:
    raise FileNotFoundError(f"No CSV files found under: {BASE_DIR}")



In [32]:
# 6) MAIN CLEANING LOOP (chunked)
seen = set()
good_chunks, bad_chunks = [], []
total_in, total_kept = 0, 0

for fp in csv_files:
    for chunk in pd.read_csv(fp, chunksize=CHUNKSIZE):
        total_in += len(chunk)
        chunk = standardize_columns(chunk)

        # Types
        if "latitude" in chunk.columns:
            chunk["latitude"] = pd.to_numeric(chunk["latitude"], errors="coerce")
        if "longitude" in chunk.columns:
            chunk["longitude"] = pd.to_numeric(chunk["longitude"], errors="coerce")
        if "month" in chunk.columns:
            chunk["month_dt"] = pd.to_datetime(chunk["month"].astype(str) + "-01", errors="coerce")

            # London forces only
        chunk, not_force = filter_forces(chunk)
        if len(not_force):
            nf = not_force.copy()
            nf["bad_reason"] = "not_london_force"
            bad_chunks.append(nf)
        if chunk.empty:
            continue

             # Fill missing
        if "last_outcome_category" in chunk.columns:
            chunk["last_outcome_category"] = chunk["last_outcome_category"].fillna("Outcome Pending")
        if "location" in chunk.columns:
            chunk["location"] = chunk["location"].fillna("Location Withheld")
        if "lsoa_code" in chunk.columns:
            chunk["lsoa_code"] = chunk["lsoa_code"].fillna("Unknown LSOA")

        # Categories & weights
        if "crime_type" in chunk.columns:
            chunk["crime_category"] = (
                chunk["crime_type"].astype(str).str.strip().str.lower().map(CATEGORY_MAP).fillna("Other")
            )
        else:
            chunk["crime_category"] = "Other"
        chunk["severity_weight"] = chunk["crime_category"].map(SEVERITY).fillna(1.0)

        # Season & temporal weight
        if "month_dt" in chunk.columns:
            month_num = chunk["month_dt"].dt.month
            season_map = {12:"Winter",1:"Winter",2:"Winter",3:"Spring",4:"Spring",5:"Spring",
                          6:"Summer",7:"Summer",8:"Summer",9:"Autumn",10:"Autumn",11:"Autumn"}
            chunk["season"] = month_num.map(season_map)
            ref = pd.to_datetime("today").normalize()
            delta = (ref - chunk["month_dt"]).dt.days.clip(lower=0)
            lam = math.log(2)/180.0
            chunk["temporal_weight"] = (-lam*delta).apply(math.exp)
        else:
            chunk["season"] = None
            chunk["temporal_weight"] = 1.0

        # Dedup
        for c in ("crime_id","month","latitude","longitude","crime_type"):
            if c not in chunk.columns:
                chunk[c] = None
        keys = (
            chunk["crime_id"].fillna("") + "|" +
            chunk["month"].fillna("") + "|" +
            chunk["latitude"].round(6).astype(str) + "|" +
            chunk["longitude"].round(6).astype(str) + "|" +
            chunk["crime_type"].fillna("")
        ).tolist()
        dup_mask = []
        for k in keys:
            if k in seen:
                dup_mask.append(True)
            else:
                seen.add(k); dup_mask.append(False)
        dup_mask = pd.Series(dup_mask, index=chunk.index)

        dups = chunk[dup_mask].copy()
        if len(dups):
            dups["bad_reason"] = "duplicate"
            bad_chunks.append(dups)

        chunk = chunk[~dup_mask]
        total_kept += len(chunk)
        good_chunks.append(chunk)

        



In [34]:
# 7) SAVE OUTPUTS
if good_chunks:
    pd.concat(good_chunks, ignore_index=True).to_csv(OUT_CLEAN, index=False)
if bad_chunks:
    pd.concat(bad_chunks, ignore_index=True, sort=False).to_csv(OUT_BAD, index=False)

print("Cleaning complete!")
print(f"  Source folder : {BASE_DIR}")
print(f"  CSV files read: {len(csv_files)}")
print(f"  Rows in       : {total_in}")
print(f"  Rows kept     : {total_kept}")
print(f"→ Clean data    : {Path(OUT_CLEAN).resolve()}")
print(f"→ Excluded rows : {Path(OUT_BAD).resolve()}")


Cleaning complete!
  Source folder : C:\Users\hksai\OneDrive\Documents\Group_Porject_Doc\Data_Police_UK_ALL(2025)\2025-08
  CSV files read: 41
  Rows in       : 516837
  Rows kept     : 87594
→ Clean data    : C:\Users\hksai\london_crime_clean.csv
→ Excluded rows : C:\Users\hksai\rows_excluded.csv
